In [1]:
import pandas as pd
import os
from sodapy import Socrata
from sqlalchemy import create_engine


In [2]:
# 1. Create a DataFrame Covid data(full csv). 
data_df = pd.read_csv('/Users/benjaminshelburn/Desktop/Classwork/Final Project/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv')

# # # 2. Create a DF from small API sample - limit 250K?
# client = Socrata("data.cdc.gov", None)
# data = client.get("n8mc-b4w4")
# data_df = pd.DataFrame.from_records(data)
# data_df

# # 3. https://stackoverflow.com/questions/66345379/not-getting-full-data-from-api-to-jupyter-notebook
# r = requests.get("https://pokeapi.co/api/v2/pokemon/")
# x = r.json()

# data = []
# data.append(x['results'])

# # Loop until `next` is `None`
# while x.get('next'):
#     r = requests.get("https://pokeapi.co/api/v2/pokemon/")
#     x = r.json()
#     data.append(x['results'])

# df = pd.DataFrame(data)

/Users/benjaminshelburn/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Columns to drop
data_df = data_df.drop(['state_fips_code'], axis=1)
data_df = data_df.drop(['county_fips_code'], axis=1)
data_df = data_df.drop(['case_positive_specimen_interval'], axis=1)
data_df = data_df.drop(['current_status'], axis=1)
data_df = data_df.drop(['case_onset_interval'], axis=1)

# Print df
data_df

,case_month,res_state,res_county,age_group,sex,race,ethnicity,process,exposure_yn,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2020-12,AR,BOONE,0 - 17 years,NaN,NaN,NaN,Missing,Missing,Symptomatic,Missing,Missing,NaN,NaN
1,2021-10,MO,PHELPS,0 - 17 years,NaN,NaN,NaN,Missing,Missing,Missing,Unknown,Missing,Unknown,NaN
2,2020-10,MO,CASS,0 - 17 years,Female,NaN,NaN,Missing,Missing,Symptomatic,Unknown,Missing,Unknown,NaN
3,2021-08,TN,CLAIBORNE,0 - 17 years,Female,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,NaN
4,2020-11,GA,EFFINGHAM,0 - 17 years,Female,Unknown,Missing,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37532067,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN
37532068,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN
37532069,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN
37532070,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN


In [4]:
# Change and format column names
data_df.rename(columns={'case_month':'Case Month', 'res_state':'Resident State', 'res_county': 'Resident County',
                       'age_group':'Age Group', 'sex':'Sex', 'race':'Race', 'ethnicity': 'Ethnicity', 
                       'process':'Process', 'exposure_yn': 'Exposure', 'symptom_status': 'Symptom Status',
                       'hosp_yn':'Hospitalization', 'icu_yn': 'ICU', 'death_yn':'Death',
                        'underlying_conditions_yn': 'Underlying Health Conditions'},inplace=True )

In [5]:
# Copy df and insert blank Final Outcome column
data_df = data_df.copy()
data_df['Final Outcome']=""
data_df

,Case Month,Resident State,Resident County,Age Group,Sex,Race,Ethnicity,Process,Exposure,Symptom Status,Hospitalization,ICU,Death,Underlying Health Conditions,Final Outcome
0,2020-12,AR,BOONE,0 - 17 years,NaN,NaN,NaN,Missing,Missing,Symptomatic,Missing,Missing,NaN,NaN,
1,2021-10,MO,PHELPS,0 - 17 years,NaN,NaN,NaN,Missing,Missing,Missing,Unknown,Missing,Unknown,NaN,
2,2020-10,MO,CASS,0 - 17 years,Female,NaN,NaN,Missing,Missing,Symptomatic,Unknown,Missing,Unknown,NaN,
3,2021-08,TN,CLAIBORNE,0 - 17 years,Female,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,NaN,
4,2020-11,GA,EFFINGHAM,0 - 17 years,Female,Unknown,Missing,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37532067,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532068,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532069,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532070,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN,


In [6]:
# remove rows using the drop() function
data_df.drop(data_df.index[data_df['Resident State'] == 'Missing'], inplace=True)
data_df.drop(data_df.index[data_df['Resident County'] == 'Missing'], inplace=True)
data_df.drop(data_df.index[data_df['Age Group'] == 'Missing'], inplace=True)
data_df.drop(data_df.index[data_df['Sex'] == 'Missing'], inplace=True)
data_df.drop(data_df.index[data_df['Race'] == 'Missing'], inplace=True)
data_df.drop(data_df.index[data_df['Ethnicity'] == 'Missing'], inplace=True)

# display the dataframe
data_df

,Case Month,Resident State,Resident County,Age Group,Sex,Race,Ethnicity,Process,Exposure,Symptom Status,Hospitalization,ICU,Death,Underlying Health Conditions,Final Outcome
0,2020-12,AR,BOONE,0 - 17 years,NaN,NaN,NaN,Missing,Missing,Symptomatic,Missing,Missing,NaN,NaN,
1,2021-10,MO,PHELPS,0 - 17 years,NaN,NaN,NaN,Missing,Missing,Missing,Unknown,Missing,Unknown,NaN,
2,2020-10,MO,CASS,0 - 17 years,Female,NaN,NaN,Missing,Missing,Symptomatic,Unknown,Missing,Unknown,NaN,
5,2020-12,OH,HOLMES,0 - 17 years,Female,NaN,NaN,Clinical evaluation,Yes,Symptomatic,No,No,No,NaN,
6,2021-04,NY,LIVINGSTON,0 - 17 years,Female,NaN,NaN,Missing,Missing,Missing,Missing,Missing,No,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37532067,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532068,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532069,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN,
37532070,2020-11,SD,NaN,65+ years,Male,White,Unknown,Missing,Missing,Symptomatic,Missing,Missing,Missing,NaN,


In [ ]:
data_df.to_csv(r'/Users/benjaminshelburn/Desktop/Classwork/Final Project/CleanData.csv', index = False)
